In [7]:
import numpy as np
import requests
import pandas as pd
import scipy
import json
import urllib
import os

In [2]:
musicBrainzURL =  "https://musicbrainz.org/ws/2/"
transcription_data = pd.read_csv("../BillboardTranscriptionData.tsv", sep="\t")
sample_data = pd.read_csv("../BillboardSampleData.tsv", sep="\t")

In [3]:
sample_data.columns

Index(['FileName', 'ARTIST', 'TITLE', 'Chart Date', 'Peak Position',
       'Weeks on Chart', 'Target Rank', 'Actual Rank', 'Sample ID#',
       'YoutubeLink', 'OriginalAudio'],
      dtype='object')

In [13]:
files = os.listdir("../Humdrum/CompleteTranscriptions/")
file_names = [os.path.splitext(file)[0] for file in files]

In [19]:
transcription_data['FileName']

0                  ATasteOfHoney_Sukiyaki_1981
1             AaronNeville_TellItLikeItIs_1966
2                         ABBA_Chiquitita_1979
3                           ABBA_Fernando_1976
4                         ABBA_HoneyHoney_1974
                        ...                   
734    WilsonPickett_SoulDanceNumberThree_1967
735                         Yaz_Situation_1982
736                     YoungMC_BustAMove_1990
737          YvonneElliman_IfICantHaveYou_1978
738                        ZZTop_LaGrange_1974
Name: FileName, Length: 739, dtype: object

In [23]:
[file_name if file_name in sample_data["FileName"] else "" for file_name in file_names]

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',


In [4]:
requests.get(musicBrainzURL+"recording/"+"f9a74bed-c572-43da-8eb5-babc4aee8ffa").content

b'<?xml version="1.0" encoding="UTF-8"?>\n<metadata xmlns="http://musicbrainz.org/ns/mmd-2.0#"><recording id="f9a74bed-c572-43da-8eb5-babc4aee8ffa"><title>Fernando</title><length>254106</length><first-release-date>1976-03-16</first-release-date></recording></metadata>\n'

In [5]:
def lookup_metadata(track_name: str, artist_name: str, incs: str) -> dict:
    """Looks up the metadata for a listen using track name and artist name."""
    params = {
        "recording_name": track_name,
        "artist_name": artist_name,
    }
    if incs:
        params["metadata"] = True
        params["incs"] = incs
        params["type"] = "single"
    response = requests.get(
        url="https://api.listenbrainz.org/1/metadata/lookup/",
        params=params
    )
    response.raise_for_status()
    return response.json()

In [6]:
lookup_metadata(transcription_data["TITLE"][2], transcription_data["ARTIST"][2], "")

{'artist_credit_name': 'ABBA',
 'artist_mbids': ['d87e52c5-bb8d-4da8-b941-9f4928627dc8'],
 'recording_mbid': 'cf9053d1-d116-407e-a989-1f30abd6702c',
 'recording_name': 'Chiquitita',
 'release_mbid': '1097b87f-dd76-4c02-9f26-092da2016ecc',
 'release_name': 'Voulez-Vous'}

In [11]:
mbids = [lookup_metadata(title, artist, "").get("recording_mbid") for title, artist in zip(transcription_data["TITLE"], transcription_data["ARTIST"])]

KeyboardInterrupt: 

In [2]:
transcription_mbids = mbids

NameError: name 'mbids' is not defined

In [1]:
transcription_mbids

NameError: name 'transcription_mbids' is not defined

In [50]:
sample_responses = [lookup_metadata(title, artist, "") for title, artist in zip(sample_data["TITLE"], sample_data["ARTIST"])]

In [63]:
sample_responses[4]

{'artist_credit_name': 'Sting',
 'artist_mbids': ['7944ed53-2a58-4035-9b93-140a71e41c34'],
 'recording_mbid': '84adf231-3b0f-47fd-95dd-446925313f48',
 'recording_name': 'All This Time',
 'release_mbid': '9e7f72d3-361e-4eff-8b84-a7decbfdc9e7',
 'release_name': 'The Soul Cages'}

In [13]:
lookup_metadata(sample_data["TITLE"][4], sample_data["ARTIST"][4], "release")

{'artist_credit_name': 'Sting',
 'artist_mbids': ['7944ed53-2a58-4035-9b93-140a71e41c34'],
 'metadata': {'recording': {'length': 295000,
   'name': 'All This Time',
   'rels': [{'artist_mbid': '7944ed53-2a58-4035-9b93-140a71e41c34',
     'artist_name': 'Sting',
     'instrument': 'lead vocals',
     'type': 'vocal'}]}},
 'recording_mbid': '84adf231-3b0f-47fd-95dd-446925313f48',
 'recording_name': 'All This Time',
 'release_mbid': '9e7f72d3-361e-4eff-8b84-a7decbfdc9e7',
 'release_name': 'The Soul Cages'}

In [73]:
[r.get("recording_mbid") for r in sample_responses[:5]]

['ff4720af-1060-4717-8db9-94ba480de249',
 '271a72de-894a-4698-9815-97851b80a6bf',
 'a0472b00-5fe8-4a46-95db-bc8247dc4ca0',
 'f2eb4b0c-d39c-4ced-b1f4-e38cd0d73ff3',
 '84adf231-3b0f-47fd-95dd-446925313f48']